In [ ]:
import guardian_module as gm
import database_connection as dc
import pandas as pd
import unicodedata

In [26]:
# Create Database Connection to local database via the database_connection.py module
username = open('database_login.txt', 'r').readlines()[0].replace('\n', '')
password = open('database_login.txt', 'r').readlines()[1].replace('\n', '')
database = open('database_login.txt', 'r').readlines()[2].replace('\n', '')
db = dc.database(username, password, database)

guardian_key = open('guardian_key.txt', 'r').readlines()[0]
gc = gm.Guardian_Cursor(guardian_key)

In [ ]:
# load Data from Company Dataset
# Data source: people data labs
# insert company data into database
company_df = pd.read_csv('Data/companies_sorted.csv')

headers = ['name', 'domain', 'year founded', 'industry', 'locality', 'country', 'linkedin url', 'current employee estimate', 'total employee estimate']
company_df_select = company_df[headers].head(10000)

company_df_select['name'] = company_df_select['name'].map(lambda name: unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii'))
company_df_select = company_df_select[(company_df_select['name']!= '') & (company_df_select['name']!= ' ')]


company_df_select.drop_duplicates('name', keep='first', inplace=True)

column_names = [x.replace(' ', '_') for x in headers]
company_df_select.columns = column_names

company_df_select.to_csv('Data/companies_selected.csv', index=False)
db.append(company_df_select, 'company')

In [ ]:
# Method to search articles for companies and add them to the database
def get_insert(search_input):
    for company in search_input['companies']:
        print('company: ' + company)
        for year in search_input['years']:
            print('year: ' + str(year))
            date_from = str(year) + '-01-01'
            date_to = str(year) + '-12-31'
            df_article, df_authors, df_author_links, df_search, df_search_links = gc.complete_search(company, date_from, date_to)

            if not df_article is None:
                db.append(df_article, 'article')         
                db.append(df_authors, 'person')
                db.append(df_author_links, 'written_by')      
                db.append(df_search, 'search')
                db.append(df_search_links, 'article_search')
            else:
                db.append(df_search, 'search')
            

            df_company_search = pd.DataFrame()
            df_company_search['search_id'] = df_search.search_id
            df_company_search['company_name'] = company
            db.append(df_company_search, 'company_search')

In [ ]:
# Testing above method

search_input = {'companies':['cern'], 'years':[2020]}
get_insert(search_input)

In [ ]:
cursor = db.get_cursor()
cursor.execute("""SELECT name 
    FROM company 
    ORDER BY current_employee_estimate DESC
    LIMIT 50
    """)

companies = [x[0] for x in cursor.fetchall()]

removing = ['tata consultancy services', 'cognizant technology solutions', 'at&t', 
    'education nationale', 'department of veterans affairs', 'unitedhealth group', 'united states postal service',
    'wipro technologies', ]

# Main reason to remove these companies is the composition of their name.
# The "Problem" here lays in the search algorithm of the Guardian API

companies = [x for x in companies if x not in removing]
companies[:1]

In [ ]:
# Get selection of companies to search articles for
cursor = db.get_cursor()
cursor.execute("""SELECT name 
    FROM company 
    ORDER BY current_employee_estimate DESC
    LIMIT 50
    """)

companies = [x[0] for x in cursor.fetchall()]

removing = ['tata consultancy services', 'cognizant technology solutions', 'at&t', 
    'education nationale', 'department of veterans affairs', 'unitedhealth group', 'united states postal service',
    'wipro technologies', ]

# Main reason to remove these companies is the composition of their name.
# The "Problem" here lays in the search algorithm of the Guardian API

companies = [x for x in companies if x not in removing]

search_input = {'companies':companies[:1], 'years':list(range(2010, 2023))}

get_insert(search_input)

In [6]:
df = pd.read_csv('temp.csv')

In [23]:
def split_dates(date):
    split_month = int(date[5:7])/2

    date_1 = date[:5] + '0' + str(split_month)[:1] + '-31'
    date_2 = date[:5] + '0' + str(split_month+1)[:1] + '-01'
    return(date_1, date_2)

date_to = '2010-12-31'

print(split_dates(date_to))

('2010-06-31', '2010-07-01')


In [42]:
df = pd.read_csv('temp.csv', index_col='Unnamed: 0')
df['length'] = [len(x) for x in df.text]
df = df[df.length <= 60000]
df = df.drop(columns=['length'])
df

,article_id,section,title,web_url,date,text
57,world/2010/jan/07/british-troops-iran-border-h...,World news,British troops tried to rescue hostages at Ira...,https://www.theguardian.com/world/2010/jan/07/...,2010-01-07T20:41:34Z,British troops in southern Iraq were scrambled...
58,world/2010/mar/05/armenia-deportations-first-w...,World news,Background: the US Turkey-Armenia 'genocide' r...,https://www.theguardian.com/world/2010/mar/05/...,2010-03-05T19:36:07Z,What happened to the Armenians in 1915? During...
59,environment/2010/feb/23/terminator-carp-threat...,Environment,'Terminator' carp threatens Great Lakes,https://www.theguardian.com/environment/2010/f...,2010-02-23T18:36:56Z,The fight looks utterly unequal. In the red co...
60,world/2010/mar/05/history-armenia-turkey-genocide,World news,Grim history of Armenians in Turkey that led t...,https://www.theguardian.com/world/2010/mar/05/...,2010-03-05T12:11:07Z,Armenia believes Turkey committed genocide in ...
61,uk/2010/feb/16/raf-chief-backs-power-in-skies,UK news,RAF chief backs power in the skies,https://www.theguardian.com/uk/2010/feb/16/raf...,2010-02-16T01:08:41Z,The head of the RAF yesterday joined the incre...
...,...,...,...,...,...,...
153,commentisfree/2010/jul/08/afghanistan-catastro...,Opinion,Afghanistan is a catastrophe. But we will have...,https://www.theguardian.com/commentisfree/2010...,2010-07-08T21:30:21Z,As British troops retreat from the fortress of...
154,commentisfree/2010/may/14/afghanistan-kandahar...,Opinion,Swift Afghanistan exit not on the cards | Robe...,https://www.theguardian.com/commentisfree/2010...,2010-05-14T15:30:00Z,"So, Afghanistan is back on the agenda. No long..."
155,film/2010/mar/08/oscars-2010-kathryn-bigelow-w...,Film,Oscars 2010: is Kathryn Bigelow's victory a wi...,https://www.theguardian.com/film/2010/mar/08/o...,2010-03-08T18:15:00Z,"After every Oscar ceremony, observers traditio..."
156,uk/2010/apr/23/military-afghanistan,UK news,Soldier describes directing fatal air strike b...,https://www.theguardian.com/uk/2010/apr/23/mil...,2010-04-23T16:01:12Z,A British soldier today choked back tears as h...
